# Feature Selection

The following code is used for feature selection using wrapper based feature selection mechanism where SVM with rbf kernel is used as a classifier. Feature selection and backward elimination both techquies are used. top_feature variable is used for selecting the different number of opcodes. It finds out that top 116 features help better in classification. Following is the code where we get features.  

Importing libraries and initializing variables

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
path_tr='nn_500_data_to_tra.csv'
path_futr='nn_500_data_to_traFull.csv'
path_te='nn_500_data_to_test.csv'
path_val='nn_500_data_to_val.csv'
top_feature=116

Reading data

In [2]:
train = np.genfromtxt(path_tr, delimiter=",", dtype=str)
val = np.genfromtxt(path_val, delimiter=",", dtype=str)

Data processing

In [3]:
x_data  = train[1:,0:top_feature]
x_label = train[1:,-1]
x_data=x_data.astype(np.float)
x_label=x_label.astype(np.int)
# Binarize the output
x_label = label_binarize(x_label, classes=[1,2,3,4,5,6,7,8,9])

y_data  = val[1:,0:top_feature]
y_data=y_data.astype(np.float)
y_label = val[1:,-1]
y_label=y_label.astype(np.int)
woby=y_label
# Binarize the output
y_label = label_binarize(y_label, classes=[1,2,3,4,5,6,7,8,9])

Applying custom grid search. optimize c and g values store in opt_c,opt_g respectively

In [4]:
c=[0.01,0.1,1,10,100]
g=[0.01,0.1,1,10,100]
loss=100
opt_c=0
opt_g=0
for i in c:
    for j in g:    
        clf = OneVsRestClassifier(svm.SVC(kernel='rbf',C=i,gamma=j, probability=True))
        clf_f = clf.fit(x_data, x_label)
        y_score=clf_f.decision_function(y_data)
        y_predict_prob=clf_f.predict_proba(y_data)
        y_predict=clf_f.predict(y_data)
        pre=y_predict.argmax(1)
        pre=pre+1
        a=log_loss(y_label,y_predict_prob)
        if a<loss:
            opt_c=i
            opt_g=j
            loss=a
        print(confusion_matrix(woby,pre))
        print('log loss: ',a,'  C: ',i,'  G: ',j)

Reading data from train full data and test data

In [5]:
train = np.genfromtxt(path_futr, delimiter=",", dtype=str)
test = np.genfromtxt(path_te, delimiter=",", dtype=str)

Data processing

In [6]:
x_data  = train[1:,0:top_feature]
x_label = train[1:,-1]
x_data=x_data.astype(np.float)
x_label=x_label.astype(np.int)
# Binarize the output
x_label = label_binarize(x_label, classes=[1,2,3,4,5,6,7,8,9])


y_data  = test[1:,0:top_feature]
y_data=y_data.astype(np.float)

y_label = test[1:,-1]
y_label=y_label.astype(np.int)
woby=y_label
# Binarize the output
y_label = label_binarize(y_label, classes=[1,2,3,4,5,6,7,8,9])

Training the classifier(SVM) on optimized parameters

In [7]:
clf = OneVsRestClassifier(svm.SVC(kernel='rbf',C=opt_c,gamma=opt_g, probability=True))
clf_f = clf.fit(x_data, x_label)

Calculating the log_loss and confusion matrix

In [8]:
y_score=clf_f.decision_function(y_data)
y_predict_prob=clf_f.predict_proba(y_data)
y_predict=clf_f.predict(y_data)
pre=y_predict.argmax(1)
pre=pre+1

a=log_loss(y_label,y_predict_prob)

print(confusion_matrix(woby,pre)) 

print('log loss: ',a,'  C: ',opt_c,'  G: ',opt_g)